# Entanglement: Basic

One of the basic concept in entanglemnt theory is what is called entangled state and what is not. For the latter case, we call it separable states

$$ \rho_{AB}=\sum_i p_i |\psi_{A,i}\psi_{B,i}\rangle \langle \psi_{A,i}\psi_{B,i}| $$

$$ p_i\geq 0,\sum_i{p_i}=1, \langle \psi_{A,i}|\psi_{A,i}\rangle=\langle \psi_{B,i}|\psi_{B,i}\rangle=1 $$

or equivalently

$$ \rho_{AB}=\sum_i p_i \sigma_{A,i}\otimes \sigma_{B,i} $$

$$ p_i\geq 0,\sum_i{p_i}=1, \sigma_{A,i}\succeq 0, \sigma_{B,i}\succeq 0,\mathrm{Tr}[\sigma_{A,i}]=\mathrm{Tr}[\sigma_{B,i}]=1 $$

which makes a convex compact closed set in the space of density matrices, which we will call it SEP. Apparently, the maximally mixed state is a separable state

$$ \rho_0 = \sum_i \frac{1}{d} | ii \rangle\langle ii |.$$

And its complement is the set of entangled states, which we will call it ENT.

$$ \mathrm{ENT}=\{ \rho\in\mathbb{C}^{d\times d}: \rho\succeq 0,\mathrm{Tr}[\rho]=1,\rho\notin \mathrm{SEP} \} $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import numqi


## Pure state

We have simple criterion of entanglement detection for pure state, simply check its rank. If the rank is 1,

$$ |\psi_{AB}\rangle=|\psi_{A}\rangle \otimes |\psi_B\rangle$$

then it is a separable state, otherwise it is entangled. The computational basis state are all separable states, e.g. $|00\rangle, |01\rangle,|10\rangle, |11\rangle$ for two qubits. Some famous entangled states are Bell states [wiki-link](https://en.wikipedia.org/wiki/Bell_state), GHZ states [wiki-link](https://en.wikipedia.org/wiki/Greenberger%E2%80%93Horne%E2%80%93Zeilinger_state), W state [wiki-link](https://en.wikipedia.org/wiki/W_state), etc.

$$ \mathrm{Bell}: |00\rangle \pm |11\rangle, |01\rangle \pm |10\rangle  $$

$$ \mathrm{GHZ}: |000\rangle + |111\rangle $$

$$ \mathrm{W}: |001\rangle + |010\rangle + |100\rangle $$

(not normalized for conciseness). Below we can calculate the rank (number of nonzero singular values) of these states (for three partite state, we simply divide into the first party and the others).

In [ ]:
state00 = np.array([1,0,0,0], dtype=np.float64)
state_bell = np.array([1, 0, 0, 1]) / np.sqrt(2)
state_ghz = np.zeros(8)
state_ghz[[0,7]] = 1/np.sqrt(2)
state_w = np.zeros(8)
state_w[[1,2,4]] = 1/np.sqrt(3)

for state in [state00, state_bell, state_ghz, state_w]:
    singular_value = np.linalg.svd(state.reshape(2,-1), compute_uv=False)
    tmp0 = 'entangled' if singular_value[1] > 1e-7 else 'separable'
    print(f'[{tmp0}] singular value: {singular_value}')


Equivalently, one can check the purity of the reduced density matrix (RDM), if the purity of the RDM is less than one, then the state is entangled. The purity of a density matrix $\rho$ is defined as [wiki-link](https://en.wikipedia.org/wiki/Purity_(quantum_mechanics))

$$ \gamma=\mathrm{Tr}[\rho^2] $$

Let's verify this criterion for the above states numerically.

In [ ]:
for state in [state00, state_bell, state_ghz, state_w]:
    tmp0 = state.reshape(2, -1)
    rdm = tmp0 @ tmp0.T.conj()
    purity = np.trace(rdm @ rdm).real
    tmp0 = 'entangled' if (1-purity>1e-7) else 'separable'
    print(f'[{tmp0}] purity: {purity}')


## PPT criterion

However, determine whether a mixed state is entangled or not is a NP-hard problem. There are some criteria that can be used to detect entanglement, e.g. PPT criteria [wiki-link](https://en.wikipedia.org/wiki/Peres%E2%80%93Horodecki_criterion). The PPT criteria is based on the fact that the partial transpose of a separable state is still a positive semidefinite matrix , while the partial transpose of an entangled state may not. The partial transpose of a matrix $\rho$ is defined as

$$ \rho\in \mathbb{C}^{d_Ad_B\times d_Ad_B}, \rho=\sum_{ijkl}\rho_{ij,kl}|ij\rangle\langle kl|,\rho^{\Gamma}=\sum_{ijkl}\rho_{il,kj} |ij\rangle\langle kl|$$

where we denote it with $\rho^{\Gamma}$. To check whether a matrix is positive semidefinite, we can check whether all its eigenvalues are nonnegative.

Let's check the partial transpose of the separable states

$$ \rho=\sum_i p_i \sigma_{A,i}\otimes \sigma_{B,i}\in\mathrm,\rho^{\Gamma}=\sum_i p_i \sigma_{A,i}\otimes \sigma_{B,i}^T $$

where $\sigma_{B,i}^T$ is the transpose of $\sigma_{B,i}$, and apparently, this partial transpose is still positive semi-definite. For the entangled states, Let's explicitly calculate the partial transpose of the Bell state

$$ \rho_{\mathrm{Bell}}=\frac{1}{2}\begin{bmatrix}1&0&0&1\\ 0&0&0&0\\ 0&0&0&0\\ 1&0&0&1\\\end{bmatrix} $$

and its partial transpose is

$$ \rho_{\mathrm{Bell}}^{\Gamma}=\frac{1}{2}\begin{bmatrix}1&0&0&0\\ 0&0&1&0\\ 0&1&0&0\\ 0&0&0&1\\\end{bmatrix} $$

with a negative eigenvalue $-1/2$. Therefore, the Bell state is entangled.

Let's check the above states with PPT criterion.

In [ ]:
for state in [state00, state_bell, state_ghz, state_w]:
    dimA = 2
    dimB = state.size//dimA
    dm = state.reshape(-1,1) * state.reshape(-1).conj()
    dm_pt = dm.reshape(dimA,dimB,dimA,dimB).transpose(0,3,2,1).reshape(dimA*dimB,-1)
    EVL = np.linalg.eigvalsh(dm_pt)
    tmp0 = 'entangled' if (EVL[0]<0) else 'separable'
    print(f'[{tmp0}] eigenvalues(PT): {EVL}')


One can also call `numqi.entangle.is_ppt` to check PPT criterion.

In [ ]:
for state in [state00, state_bell, state_ghz, state_w]:
    dimA = 2
    dimB = state.size//dimA
    dm = state.reshape(-1,1) * state.reshape(-1).conj()
    tmp0 = 'separable' if numqi.entangle.is_ppt(dm, (dimA,dimB)) else 'entangled'
    print(f'[{tmp0}]')


Let's define the set of PPT states as all states that satisfy the PPT criterion, and its complement is the set of NPT states.

$$ \mathrm{PPT} = \left\{ \rho\in\mathbb{C}^{d_Ad_B\times d_Ad_B}: \rho\succeq 0,\mathrm{Tr}[\rho]=1,\rho^{\Gamma}\succeq 0 \right\} $$

The PPT set is also convex, closed and compact. Apparently, SEP set is a subset of PPT set $\mathrm{SEP}\subseteq \mathrm{PPT}$ and ENT set is a subset of NPT set $\mathrm{NPT}\subseteq \mathrm{ENT}$. When dimension is less or equal than six $d_Ad_B\leq 6$, $\mathrm{PPT}=\mathrm{SEP}$.

In previous tutorial (TODO-link), we introduce numerical range for visualizing the geometry of the density matrix by projecting all density matrix into a two dimensional plane. We can also do this for PPT set. Let's visualize the PPT set for two qubits with operators

$$ O_1=X\otimes X,O_2=Z\otimes Z. $$

In [ ]:
op0 = np.kron(numqi.gate.X, numqi.gate.X)
op1 = np.kron(numqi.gate.Z, numqi.gate.Z)
# op1 = np.kron(numqi.gate.Z, numqi.gate.I) + np.kron(numqi.gate.I, numqi.gate.Z)

theta_list = np.linspace(0, 2*np.pi, 400)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)

beta_dm = numqi.matrix_space.get_joint_algebraic_numerical_range([op0,op1], direction)
nr_dm = beta_dm.reshape(-1,1)*direction

beta_ppt = numqi.entangle.get_ppt_numerical_range([op0, op1], direction, dim=(2,2))
nr_ppt = beta_ppt.reshape(-1,1)*direction

fig,ax = plt.subplots(figsize=(8,6))
ax.plot(nr_dm[:,0], nr_dm[:,1], label='DM') #density matrix
ax.plot(nr_ppt[:,0], nr_ppt[:,1], label='PPT')
ax.legend()
ax.set_label('$O_0$')
ax.set_label('$O_1$')


Apparently, the PPT set is within density matrix set (labeled "DM" in the figure). And for this two-qubits Hilbert space, the PPT set is equavalent to the SEP set. One can check that the PPT vertex $(0,\pm 1)$ corresponds to the density matrix

$$ \frac{1}{2}|00\rangle\langle 00| + \frac{1}{2}|11\rangle\langle 11| $$

$$ \frac{1}{2}|00\rangle\langle 00| - \frac{1}{2}|11\rangle\langle 11| $$

*Task*: can you figure out the density matrix corresponds to the other two vertices $(\pm 1, 0)$?

*Task*: What about the numerical range with respect to the operators $O_1=X\otimes X,O_2=Z\otimes I+I\otimes Z $

For larger dimension $d_Ad_B>6 (d_A\geq 2, d_B\geq 2)$, the PPT criterion is a necessary condition for separability, but not sufficient, and there is PPT state which is not separable (bounded entanglement [wiki-link](https://en.wikipedia.org/wiki/Bound_entanglement)).

Below, we give an example of PPT but not separable state, called Tiles UPB/BES (Bounded Entangled state from Unextendible Product States) in two-qutrits system $\mathcal{H}_3\otimes \mathcal{H}_3$. But this state can be detected by so-called generalized PPT criteria from paper "The generalized partial transposition criterion for separability of multipartite quantum states" [doi-link](https://doi.org/10.1016/S0375-9601(02)01538-4). In the later tutorials, we will introduce more advanced entanglement detection criteria.

In [ ]:
upb_list, rho_bes = numqi.entangle.load_upb('tiles', return_bes=True)
print('satisfy PPT criterion:', numqi.entangle.is_ppt(rho_bes, dim=(3,3)))
print('satisfy generalized PPT criterion:', numqi.entangle.is_generalized_ppt(rho_bes, dim=(3,3)))


Some good references for entanglement detection criteria see

1. [arxiv-link](https://arxiv.org/abs/quant-ph/9703004v2) Separability criterion and inseparable mixed states with positive partial transposition, by Pawel Horodecki